# Required packages

In [ ]:
# !pip install similaritymeasures
# !pip install scikit-learn-extra
# !pip install geopy

# Download the data

In [ ]:
# Step_1: Download data From: https://www.kaggle.com/datasets/akkithetechie/new-york-city-bike-share-dataset

# Data preprocessing

In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

from Bike_utils_similarity.Exp import *
from Bike_utils_similarity.counter import *
from Bike_utils_similarity.extra_utils import data_partition,write_
from Bike_utils_similarity.utils.pkl_utils import *
from Bike_utils_similarity.utils.time_utils import get_TimeStamp_str
from Bike_utils_similarity.utils.distance_extraction import get_distance
from Bike_utils_similarity.similarity_utils import similarity, get_hours_curve_data

import pandas as pd
from sklearn_extra.cluster import KMedoids
from sklearn.preprocessing import LabelEncoder

In [2]:
# --------------- 1 start
df = pd.read_csv('NYC-BikeShare-2015-2017-combined.csv')

for f in ['User Type']: # 'Start Station Name' ==> "moved commented" ==> Mon_2024_5_6__12_20_49
    le = LabelEncoder()
    df[f] = le.fit_transform(df[f])  

a = pd.to_datetime(df['Start Time'])
df['DO_week'] = a.dt.day_of_week
df['DO_month'] = a.dt.day
df['month'] = a.dt.month
df['hour'] = a.dt.hour
df['min'] = a.dt.minute
df['year'] = a.dt.year

df['age'] = df['year'] - df['Birth Year']

df['Start Hour'] = a.dt.hour 
# --------------- 1 end   

## Encoding "Start" & "End" Station_names

In [3]:
# ========== mappping_dic ===============================
# --------------- 2 start 
e = LabelEncoder()
df['Start Station Encoding'] = e.fit_transform(df['Start Station Name'])    
dd = df[['Start Station Name', 'Start Station Encoding']]
mappping_dic = dd.set_index('Start Station Name').T.to_dict('list')
# dump(mappping_dic,'mappping_dic')

for k,v in mappping_dic.items():
    mappping_dic[k] = v[0]
    
def custom_encoding(x):
    return mappping_dic[x] if x in mappping_dic.keys() else -1

df['End Station Encoding'] = df['End Station Name'].apply(lambda x:custom_encoding(x))
df = df[~(df['End Station Encoding']==-1) ]
dump(mappping_dic,'mappping_dic')
# --------------- 2 End

C:\Users\Fathalla\AppData\Local\Temp\ipykernel_24752\2461915161.py:5: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  mappping_dic = dd.set_index('Start Station Name').T.to_dict('list')


In [4]:
# --------------- 3 Start: Calc Distance
from Bike_utils_similarity.utils.distance_extraction import get_distance
df['distance'] = df[['Start Station Latitude',
                     'Start Station Longitude',
                     'End Station Latitude',
                     'End Station Longitude']].apply(lambda x: get_distance(x), axis=1)
# --------------- 3 End

# Save Modified DataFrame

In [ ]:
df.to_csv('NYC-BikeShare-2015-2017-modified.csv', index=False)

# Steps for the proposed method

In [6]:
# --------------- 4 start    "dic.csv"
count = 0
l=[]
for i in range(len(mappping_dic)):
    for j in range(i, len(mappping_dic)):
        l.append(['%d-%d'%(i,j), count])
        count+=1
dic_ = pd.DataFrame( np.array(l), columns = ['key', 'value'])     
dic_.to_csv('dic.csv', index=False)
# --------------- 4 End

In [8]:
# --------------- 5 start: Count number of trips between two stations
data_partition_ = data_partition(df.copy())

v_lst = list(mappping_dic.values())
z = np.zeros((len(v_lst), len(v_lst)), dtype=int)
mappping_dic = load_pkl('mappping_dic') # =======================================

for i in range(len(mappping_dic.values())):
    for j in range(i, len(mappping_dic.values())):
        z[i,j] = data_partition_.get_data( v_lst[i], v_lst[j], info_='shape')
        
d = pd.DataFrame(z, columns=v_lst)
d.to_csv('Trip count_martix.csv', index=False)        
# --------------- 5 End    

In [11]:
# --------------- 6 start - Generate Curves data
Conf_matrix_cell_curve_data = {}
v_lst = list(mappping_dic.values())
data_partition_ = data_partition(df.copy())

for i in range(len(mappping_dic.values())):
    for j in range(i, len(mappping_dic.values())):
        dframe = data_partition_.get_data( v_lst[i], v_lst[j], info_='data')
        Conf_matrix_cell_curve_data['%s-%s'%(i,j)] = get_hours_curve_data(dframe)
        
dump(Conf_matrix_cell_curve_data,'Conf_matrix_cell_curve_data')        
# --------------- 6 End  

In [32]:
# --------------- 7 start - "dist_matrix"
Conf_matrix_cell_curve_data = load_pkl('Conf_matrix_cell_curve_data')
k_lst = list(Conf_matrix_cell_curve_data.keys())
dist_matrix = np.zeros((len(k_lst),len(k_lst)))

similarity_ = similarity(Conf_matrix_cell_curve_data)

for i in range(len(k_lst)):
    for j in range(i+1, len(k_lst)):
        x = similarity_.calc_similarity(k_lst[i], k_lst[j]) 
        dist_matrix[i,j] = x
        dist_matrix[j,i] = x
dist_matrix = pd.DataFrame( np.array(dist_matrix) )
dist_matrix.to_csv('dist_matrix.csv', index=False)
# --------------- 7 End

In [39]:
# --------------- 8 "clustering.pkl"  KMedoids: 
dist_matrix = pd.read_csv('dist_matrix.csv',index_col=None)
res={}
for i in range(2,11):   
    model = KMedoids(n_clusters=i, random_state=0).fit(dist_matrix)
    res[str(i)]=model.labels_
dump(res, 'clustering') #    of length 1326 
# --------------- 8 End  

# Run Experiment

In [5]:
df = pd.read_csv('NYC-BikeShare-2015-2017-modified.csv')
dic_df = pd.read_csv('dic.csv')
res = load_pkl('clustering')

data_partition_ = data_partition(df.copy())

cluster_df = None

for i in range(2,3):
    
    exp_df = None
    results = []
    tmp = dic_df.copy()
    tmp['cluster'] = res[str(i)]
    cluster_id_results = []
    
    for cluster_id in set(res[str(i)]):
        cluster_df = tmp[tmp['cluster']==cluster_id]
        cluster_id_df = pd.DataFrame()
        
        for k in cluster_df['key'].values: # 
            w = [int(w) for w in k.split('-')]
            sub_df = data_partition_.get_data(w[0], w[1], info_='data')
            cluster_id_df = pd.concat([cluster_id_df, sub_df])
        
        # train and predict the data =====================================================================
        write_('output','\tcluster_df.shape ---------- cluster_id:%d'%cluster_id , cluster_df.shape, "  DF length:", cluster_id_df.shape[0] )
        exp_res = ML_exp(cluster_id_df, 
                         start_model=0, 
                         end_model=10,
                         cluster='Clustering:%d-k:%d'%(i,cluster_id))
        
        # get the results =====================================================================
        cluster_id_results.extend(exp_res)
    
    all_res = pd.DataFrame(np.array(cluster_id_results), columns=results_cols)
    all_res.to_csv('cluster %d - %s.csv'%(i,get_TimeStamp_str()), index=False)

	cluster_df.shape ---------- cluster_id:0 (1230, 3)   DF length: 601156 
	cluster_df.shape ---------- cluster_id:1 (96, 3)   DF length: 133984 


# ALL Data

In [ ]:
from Bike_utils_similarity.Exp import *
from Bike_utils_similarity.extra_utils import data_partition,write_
from Bike_utils_similarity.utils.pkl_utils import *
from Bike_utils_similarity.utils.time_utils import get_TimeStamp_str
# from Bike_utils_similarity.utils.distance_extraction import get_distance
from Bike_utils_similarity.similarity_utils import similarity, get_hours_curve_data

import pandas as pd


df = pd.read_csv('NYC-BikeShare-2015-2017-modified.csv')

exp_res = Similarity_ML_exp(df, 
                            start_model=0, 
                            end_model=10,
                            cluster='all_data')

all_res = pd.DataFrame(np.array(exp_res), columns=results_cols)
all_res.to_csv('cluster-2015-2017 all_data - %s.csv'%(get_TimeStamp_str()), index=False)